# Extract Transform Load (ETL)

Let say, I'm making an application to manage my iTunes music. 
The goal of this project is to extract my iTunes dataset in XML format.

ETL is a process that extracts the data from different source systems, then transforms the data (like applying calculations, concatenations, etc.) and finally loads the data into the Data Warehouse/ Database system.

## Data Modeling

Designed 3 tables, Artist, Album, Track.The tables are made in SQLite database which is used to read,write, retrieve the data.



In [11]:
import xml.etree.ElementTree as ET
import sqlite3

## creating DATABASE connection in SQLite database
conn = sqlite3.connect('hitesh_trackdb.sqlite')
cur = conn.cursor()

In [12]:
####### CREATE TABLES using executescript()  ########

cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;


CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);



CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    len INTEGER,count INTEGER
);
''')

In [13]:
## INPUT XML FILE OF ITUNES


                                       ###### EXTRACTING Required data from XML #######

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Music.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None


                                               ######### TRANSFORMING ##########

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    
    length = lookup(entry, 'Total Time')
    

    if name is None or artist is None or album is None: 
        continue

    print(name, artist, album, count,length)

    
                                             ####### LOADING into DATABASE #######   
    
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
    

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, count) 
        VALUES ( ?, ?, ?, ?)''', 
        ( name, album_id, length, count ) )

    conn.commit()

Enter file name: Music.xml
Dict count: 310
How Long (DJJOhAL.Com) Charlie Puth (DJJOhAL.Com) How Long (DJJOhAL.Com) 9 None 196623
Meant to Be Ft. Florida Georgia Line - Bebe Rexha(Mp3Fun.IN) Mp3Fun.IN Meant to Be Ft. Florida Georgia Line - Mp3Fun.IN 9 None 178024
Perfect [muzmo.ru] [muzmo.ru] Ed Sheeran [muzmo.ru] 22 None 260205
Stranger Things - Kygo ft. OneRepublic 320kbps(WebGana.In) WebGana.In Kygo - Stranger Things ft. OneRepublic - WebGana.In 26 None 223111
Uncover Zara Larsson Introducing EP 11 None 213185
Wolves Selena Gomez Marshmello-(Mp3Sun.Com)  Selena Gomez, English Single Track Mp3 Songs 31 None 198000
Never Forget You (Audio) Zara Larsson Howwe All Music 19 None 213498
Besharmi Ki Height - Songspk.name Benny Dayal, Shalmali Kholgade  Main Tera Hero (2014) None None 289384
Ae Dil Hai Mushkil -  DownloadMing.SE Pritam , Arijit Singh Ae Dil Hai Mushkil (2016) 4 None 269165
Bol Do Na Zara - DJMaza.Link Armaan Malik Azhar (2016) 6 None 293616
Sab Tera - DJMaza.Info Armaan Mal

Heartbeat Enrique Iglesias Ft. Nicole Scherzinger Heartbeat (Featuring Nicole Scherzinger) - Single 2 None 257097
Main Hoon Hero Tera (Armaan Malik Version) - DJMaza.Info Armaan Malik Hero (2015) 4 None 284264
Yadaan Teriyaan (Version 2) - DJMaza.Info Dev Negi, Shipra Goyal Hero (2015) 4 None 302132
How You Remind Me Nickelback Silver Side Up 5 None 223817
Tumhe Apna Banane Ka - DJMaza.Link Armaan Malik & Neeti Mohan Hate Story 3 (2015) None None 309942
Wajah Tum Ho - DJMaza.Link Armaan Malik Hate Story 3 (2015) 1 None 357720
Humdard - SongsCloud.IN Arijit Singh Ek Villain (www.SongsCloud.IN) 4 None 260336
Ishq Bhi Kiya Re Maula - www.soulofmuzic.in Ali Azmat Jism 2 None None 308558
Jigar Da Tukda - www.DJMaza.Com Salim Merchant & Shraddha Pandit Ladies VS Ricky Bahl (2011) None None 252160
Jo Tere Sang - www.Songs.PK Mustafa Zahid Blood Money None None 306233
Judai (Mp3Masty.Com) Falak (Mp3Masty.Com) Jannat 2 (Mp3Masty.Com) None None 180035
Kabhi Alvida Naa Kehna ::: www.sensongs.com 

Sajde - DownloadMing.SE Arijit Singh & Nihira Joshi Kill Dil (2014) None None 325929
Wafa Ne Bewafai -  DownloadMing.SE Arijit Singh, Neeti Mohan, Suzanne DMello , Himesh Reshammiya Teraa Suroor (2016) 1 None 320522
03 Tum Saath Ho (SongsMp3.Com) Arijit Singh (SongsMp3.Com) Tamasha (SongsMp3.Com) 10 None 341185
Jaise Mera Tu - www.songsfarm.info Arijit Singh & Priya Saraiya Happy Ending (2014) 2 None 256182
Khul Kabhi - www.songsfarm.info Arijit Singh Haider (2014) None None 368953
Dhokha Dhadi - DownloadMing.SE Arijit Singh & Palak Muchhal R... Rajkumar (2013) 1 None 248737
Hai Dil Ye Mera - DownloadMing.SE Arijit Singh Hate Story 2 (2014) None None 297926
Taake Jhanke - DownloadMing.SE Arijit Singh Queen (2014) 1 None 290142
Daayre -  DownloadMing.SE Arjit Singh Dilwale (2015) 1 None 290690
Dhuaan - DownloadMing.SE Arijit Singh & Pawni Pandey Fugly (2014) None None 220656
Love Me Thoda Aur - DownloadMing.SE Arijit Singh & Monali Thakur Yaariyan (2014) 2 None 265978
Saudebaazi - Downl